In [1]:
!pip install encodec
from encodec.utils import convert_audio
import torchaudio
import torch
import ipywidgets as widgets
from IPython.display import display, Audio

ERROR: Ignored the following versions that require a different python version: 0.0.0 Requires-Python >=3.8.0; 0.1.0 Requires-Python >=3.8.0; 0.1.1 Requires-Python >=3.8.0
ERROR: Could not find a version that satisfies the requirement encodec (from versions: none)
ERROR: No matching distribution found for encodec


In [2]:
from transformers import AutoProcessor, EncodecModel, EncodecConfig
processor = AutoProcessor.from_pretrained("facebook/encodec_24khz")
        
model = EncodecModel.from_pretrained("facebook/encodec_24khz")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


EncodecModel(
  (encoder): EncodecEncoder(
    (layers): ModuleList(
      (0): EncodecConv1d(
        (conv): Conv1d(1, 32, kernel_size=(7,), stride=(1,))
      )
      (1): EncodecResnetBlock(
        (block): ModuleList(
          (0): ELU(alpha=1.0)
          (1): EncodecConv1d(
            (conv): Conv1d(32, 16, kernel_size=(3,), stride=(1,))
          )
          (2): ELU(alpha=1.0)
          (3): EncodecConv1d(
            (conv): Conv1d(16, 32, kernel_size=(1,), stride=(1,))
          )
        )
        (shortcut): EncodecConv1d(
          (conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
        )
      )
      (2): ELU(alpha=1.0)
      (3): EncodecConv1d(
        (conv): Conv1d(32, 64, kernel_size=(4,), stride=(2,))
      )
      (4): EncodecResnetBlock(
        (block): ModuleList(
          (0): ELU(alpha=1.0)
          (1): EncodecConv1d(
            (conv): Conv1d(64, 32, kernel_size=(3,), stride=(1,))
          )
          (2): ELU(alpha=1.0)
          (3): EncodecC

In [3]:
file_path = "./eval/evaluate.wav"
original_audio, original_sampling_rate = torchaudio.load(file_path)

In [4]:
original_audio, original_sampling_rate

(tensor([[ 0.0000,  0.0000,  0.0000,  ..., -0.0620, -0.0593, -0.0547],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.0573, -0.0562, -0.0539]]),
 24000)

In [5]:
audio_np = original_audio.squeeze().numpy()

# Ensure that the audio has a single channel
if audio_np.ndim > 1:
    audio_np = audio_np[0]


audio_widget = Audio(data=audio_np, rate=original_sampling_rate)
display(audio_widget)

In [9]:
# Move inputs to the same device as the model
inputs = processor(audio_np, sampling_rate=original_sampling_rate, return_tensors="pt")
inputs = {key: value.to(device) for key, value in inputs.items()}

print(inputs["input_values"].shape)

# explicitly encode then decode the audio inputs
encoder_outputs = model.encode(inputs["input_values"], inputs["padding_mask"], 3)
print(encoder_outputs[0][0].shape, type(encoder_outputs))

torch.Size([1, 1, 240000])
torch.Size([1, 4, 750]) <class 'transformers.models.encodec.modeling_encodec.EncodecEncoderOutput'>


In [8]:
print('total seconds', encoder_outputs.audio_codes.size(0))

total seconds 1


###### decode

In [11]:
print(encoder_outputs.audio_codes.shape, encoder_outputs.audio_scales)

audio_values = model.decode(encoder_outputs.audio_codes, encoder_outputs.audio_scales)[0]
print(audio_values.shape)
# Convert audio tensor to numpy array
audio_np = audio_values.cpu().squeeze().detach().numpy()
print(audio_np.shape)
audio_widget = Audio(data=audio_np, rate=processor.sampling_rate)
display(audio_widget)

torch.Size([1, 1, 4, 750]) [None]
torch.Size([1, 1, 240000])
(240000,)


In [13]:
processor.sampling_rate

48000

In [14]:
encoder_outputs.audio_codes.shape

torch.Size([47, 1, 2, 150])